In [168]:
#TO DO

# 1. Implementar ROC al hacer CrossValidation
# 2. Implementar las demas metricas para el mejor modelo de cada clase
# 3. Revisar CrossValidation en DecisionTree y NaiveBayes
# 4. Revisar errores en SVM Lineal y LogisticRegression
# 5. Completar preprocesamientos (valores nulos, encoding, scaling, seleccion)
# 6. Revisar y completar la parte de ensambles
# 7. Implementar Red Neuronal
# 8. Implementar Cascading

# 9. Separar modelos en respectivos notebooks
# 10. Predecir sobre los datos nuevos

In [167]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
import matplotlib.gridspec as gridspec

from sklearn import neighbors, datasets
from sklearn import naive_bayes
from sklearn import svm
from sklearn import tree

from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn import model_selection
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

from sklearn.metrics import roc_curve, auc
    

# Datos

In [2]:
data = pd.read_csv('tp-2020-2c-train-cols2.csv')
targets = pd.read_csv('tp-2020-2c-train-cols1.csv')

In [3]:
df = (pd.merge(left=data, right=targets, how="left", on="id_usuario", validate="one_to_one"))

In [4]:
df.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede,volveria
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes,0
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes,0
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita,0
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo,0
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo,1


# Preprocesamiento

In [5]:
encoding_params = {'onehot_drop':'first'
                  }
scaling_params = {'standard_withmean':True}
selection_params = {'vt_threshold':0,
                    'rfe_estimator':'estimator'}

columnas_a_encodear = ['tipo_de_sala', 'genero', 'nombre_sede']

In [6]:
#encoders
def OrdinalEncoderWrapper(encoding_params):
    return OrdinalEncoder()

def OneHotWrapper(encoding_params):
    return OneHotEncoder(drop=encoding_params['onehot_drop'])

def LabelEncoderWrapper(encoding_params):
    return LabelEncoder()

encoders = {'ordinal': OrdinalEncoderWrapper(encoding_params),
           'label': LabelEncoderWrapper(encoding_params),
            'onehot': OneHotWrapper(encoding_params)
           }

#scalers
def StandardScalerWrapper(scaling_params):
    return StandardScaler(with_mean=scaling_params['standard_withmean'])

def MinMaxScalerWrapper(scaling_params):
    return MinMaxScaler()

def RobustScalerWrapper(scaling_params):
    return RobustScaler()

def PowerTransformerWrapper(scaling_params):
    return PowerTransformer()

def NormalizerWrapper(scaling_params):
    return Normalizer()

scalers = {'standard': StandardScalerWrapper(scaling_params),
          'minmax': MinMaxScalerWrapper(scaling_params),
          'robust': RobustScalerWrapper(scaling_params),
           'power' : PowerTransformerWrapper(scaling_params),
           'normalizer' : NormalizerWrapper(scaling_params)
          }

#Selectors


def VarianceThresholdWrapper(selection_params):
    return VarianceThreshold()

def RFEWrapper(selection_params):
    return RFE(selection_params['rfe_estimator'])

def FeatureHasherWrapper(selection_params):
    return FeatureHasher()


selectors = {'var_thres': VarianceThresholdWrapper(selection_params),
            'rfe': RFEWrapper(selection_params),
            'feature_hasher': FeatureHasherWrapper(selection_params)}

In [126]:
#Funciones auxiliares
#robado de la practica
def droppear_nulos_por_columna(data):
    NULL_REMOVE_PCT = 0.30
    cols = data.isna().mean()
    cols = cols[cols < NULL_REMOVE_PCT]
    return data[cols.index]

def droppear_filas_sin_sede(data):
    _data = data.drop(data.loc[data['nombre_sede'].isna()].index, inplace=False)
    _data.reset_index(drop=True)
    return _data
    

def prepro_1(X, y, encoder, scaler, selector):
    _X = X.copy(deep=True)
    _y = y.copy(deep=True)
    
    #tratamiento de nulos
    _X = droppear_nulos_por_columna(_X)
    #_X = droppear_filas_sin_sede(_X) rompe el dataframe, no se por que
    _X.drop(['id_ticket','nombre','id_usuario'], axis=1, inplace=True)
    _X['edad'] = SimpleImputer(strategy='median').fit_transform(_X[['edad']])
    
    
    #encoding
    _encoder = encoders[encoder]
    if(encoder == 'onehot'):
        for col in columnas_a_encodear:
            
            encoded = _encoder.fit(_X[[col]].astype(str))
            categories = list(encoded.categories_)
            encoded = encoded.transform(_X[[col]].astype(str)).todense().astype(int)
            encoded = pd.DataFrame(encoded)
            categories = np.delete(categories, 0)
            encoded.columns = categories
            
            _X = pd.concat([_X, encoded], axis=1)
            _X.drop(labels=col, axis=1, inplace=True)
    
    #seleccion
    _selector = selectors[selector]
    #_X = _selector.fit_transform(_X,_y)
    
    #train-test split
    X_train, X_test, y_train, y_test = model_selection.train_test_split(_X, _y, test_size=0.2, random_state=7, stratify=y['volveria'])
    
    #escalado
    if(scaler != 'none'):
        _scaler = scalers[scaler]
        X_train = _scaler.fit_transform(X_train, y_train)
        X_test = _scaler.transform(X_test)
    
    y_train = y_train['volveria'].to_numpy(copy=True)
    y_test = y_test['volveria'].to_numpy(copy=True)
    
    return X_train, X_test, y_train, y_test


In [127]:
X_train_a, X_test_a, y_train_a, y_test_a = prepro_1(data, targets, 'onehot', 'standard', 'var_thres')
X_train_b, X_test_b, y_train_b, y_test_b = prepro_1(data, targets, 'onehot', 'minmax', 'var_thres')
X_train_c, X_test_c, y_train_c, y_test_c = prepro_1(data, targets, 'onehot', 'robust', 'var_thres')
X_train_d, X_test_d, y_train_d, y_test_d = prepro_1(data, targets, 'onehot', 'power', 'var_thres')
X_train_e, X_test_e, y_train_e, y_test_e = prepro_1(data, targets, 'onehot', 'normalizer', 'var_thres')

datos = [
    [X_train_a, X_test_a, y_train_a, y_test_a, 'onehot', 'standard', 'var_thres'],
    [X_train_b, X_test_b, y_train_b, y_test_b, 'onehot', 'minmax', 'var_thres'],
    [X_train_c, X_test_c, y_train_c, y_test_c, 'onehot', 'robust', 'var_thres'],
    [X_train_d, X_test_d, y_train_d, y_test_d, 'onehot', 'power', 'var_thres'],
    [X_train_e, X_test_e, y_train_e, y_test_e, 'onehot', 'normalizer', 'var_thres']
]

---

# Entrenando modelos

In [166]:
def aplicar_gscv(gscv, columnas):
    metricas = []
    for conjunto in datos:
    
        gs_results = gscv.fit(conjunto[0], conjunto[2]) #X_train e y_train, respectivamente
        row = []
        row.append(conjunto[4]) #encoding
        row.append(conjunto[5]) #scaling
        row.append(conjunto[6]) #selection
        
        for key in gs_results.best_params_:
            row.append(gs_results.best_params_[key])
        row.append(gs_results.best_score_)
        row.append(gs_results.score(conjunto[1], conjunto[3]))
        metricas.append(row)
    
    resultados = pd.DataFrame(metricas, columns=columnas)
    display(resultados)

### 3.1 KNN

Entrenamiento con valores por defecto.

In [150]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_knn = neighbors.KNeighborsClassifier()
    clf_knn.fit(conjunto[0], conjunto[2])
    pred = clf_knn.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.801242
1,onehot,minmax,var_thres,0.801242
2,onehot,robust,var_thres,0.795031
3,onehot,power,var_thres,0.838509
4,onehot,normalizer,var_thres,0.788820


---

Busqueda de hiperparámetros mediante GridSearch

In [151]:
clf_knn = neighbors.KNeighborsClassifier()

knn_grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights' : ['distance', 'uniform'],
    'metric' : ['euclidean', 'manhattan']
}

gs = model_selection.GridSearchCV(
    clf_knn,
    knn_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1,
    scoring='accuracy'
)

columnas=['encoding','scaling','selection','metric', 'n_neighbors', 'weights', 'score', 'holdout_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


,encoding,scaling,selection,metric,n_neighbors,weights,score,holdout_score
0,onehot,standard,var_thres,manhattan,11,uniform,0.828125,0.770186
1,onehot,minmax,var_thres,euclidean,3,uniform,0.815625,0.819876
2,onehot,robust,var_thres,manhattan,19,uniform,0.825000,0.795031
3,onehot,power,var_thres,manhattan,11,uniform,0.817187,0.801242
4,onehot,normalizer,var_thres,manhattan,11,distance,0.803125,0.770186


---

### 3.2 Naive Bayes

#### 3.2.1 Gaussiano

In [131]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_nb_gauss = naive_bayes.GaussianNB()
    clf_nb_gauss.fit(conjunto[0], conjunto[2])
    pred = clf_nb_gauss.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.645963
1,onehot,minmax,var_thres,0.658385
2,onehot,robust,var_thres,0.652174
3,onehot,power,var_thres,0.614907
4,onehot,normalizer,var_thres,0.664596


---

#### 3.2.2 Multinomial

In [21]:
#StandardScaler, RobustScaler y PowerTransformer arrojan valores negativos, no podemos usar el resultado en MultinomialNB

In [132]:
#Datos escalados con MinMaxScaler
nb_mult_alpha = 1.0
clf_nb_mult = naive_bayes.MultinomialNB()
clf_nb_mult.fit(X_train_b, y_train_b)
pred = clf_nb_mult.predict(X_test_b)
score = (y_test_b == pred)
score.mean()

0.7453416149068323

In [133]:
#Datos escalados con Normalizer
nb_mult_alpha = 1.0
clf_nb_mult = naive_bayes.MultinomialNB()
clf_nb_mult.fit(X_train_e, y_train_e)
pred = clf_nb_mult.predict(X_test_e)
score = (y_test_e == pred)
score.mean()

0.6149068322981367

#### 3.2.3 Complemento

In [24]:
#No se vio en clase

#### 3.2.4 Bernoulli

In [25]:
#No se vio en clase

### 3.3 SVM

#### 3.3.1 Lineal

In [134]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_linear_svc = svm.LinearSVC()
    clf_linear_svc.fit(conjunto[0], conjunto[2])
    pred = clf_linear_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.788820
1,onehot,minmax,var_thres,0.788820
2,onehot,robust,var_thres,0.788820
3,onehot,power,var_thres,0.807453
4,onehot,normalizer,var_thres,0.695652


In [135]:
#Datos escalados con StandardScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_a, y_train_a)
pred = clf_linear_svc.predict(X_test_a)
score = (y_test_a == pred)
score.mean()

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7888198757763976

In [136]:
#Datos escalados con MinMaxScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_b, y_train_b)
pred = clf_linear_svc.predict(X_test_b)
score = (y_test_b == pred)
score.mean()

0.7888198757763976

In [111]:
#Datos escalados con RobustScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_c, y_train_c)
pred = clf_linear_svc.predict(X_test_c)
score = (y_test_c == pred)
score.mean()

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7530864197530864

In [137]:
#Datos escalados con PowerTransformer

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_d, y_train_d)
pred = clf_linear_svc.predict(X_test_d)
score = (y_test_d == pred)
score.mean()

0.8074534161490683

In [138]:
#Datos escalados con Normalizer

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_e, y_train_e)
pred = clf_linear_svc.predict(X_test_e)
score = (y_test_e == pred)
score.mean()

0.6956521739130435

---

Busqueda de hiperparametros mediante GridSearch

In [152]:
clf_linear_svc = svm.LinearSVC()

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'penalty' : ['l1', 'l2']
}

gs = model_selection.GridSearchCV(
    clf_linear_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'penalty', 'score', 'holdout_score']
aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.80625         nan 0.80625         nan 0.80625         nan
 0.8046875]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.8203125       nan 0.803125        nan 0.80625         nan
 0.8046875]
  category=UserWarning


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.809375        nan 0.8078125       nan 0.8078125       nan
 0.8046875]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.79375         nan 0.803125        nan 0.803125        nan
 0.8046875]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.6203125       nan 0.6765625       nan 0.7234375       nan
 0.7640625]
  category=UserWarning


,encoding,scaling,selection,C,penalty,score,holdout_score
0,onehot,standard,var_thres,0.01,l2,0.806250,0.788820
1,onehot,minmax,var_thres,0.01,l2,0.820312,0.801242
2,onehot,robust,var_thres,0.01,l2,0.809375,0.795031
3,onehot,power,var_thres,10.00,l2,0.804688,0.801242
4,onehot,normalizer,var_thres,10.00,l2,0.764062,0.745342


---

#### 3.3.2 Polinomial

Entrenamiento con valores por defecto

In [116]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='poly')
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.790123
1,onehot,minmax,var_thres,0.790123
2,onehot,robust,var_thres,0.728395
3,onehot,power,var_thres,0.790123
4,onehot,normalizer,var_thres,0.654321


---

Busqueda de hiperparametros mediante GridSearch

In [38]:
clf_svc = svm.SVC(kernel='poly')

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'degree': [2,3,4],
    'coef0': [0.0, 0.5, 1.0],
    'gamma': [1.0, 1.5, 2.0]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)



In [83]:
#Tardan mucho por la cantidad de fits que hay que hacer

In [ ]:
gs_results = gs.fit(X_train_a, y_train_a)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
gs_results = gs.fit(X_train_b, y_train_b)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_c, y_train_c)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_d, y_train_d)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_e, y_train_e)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

---

#### 3.3.3 Radial

Entrenamiento con valores por defecto

In [34]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='rbf')
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.804979
1,onehot,minmax,var_thres,0.800830
2,onehot,robust,var_thres,0.809129
3,onehot,power,var_thres,0.809129
4,onehot,normalizer,var_thres,0.672199


---

Busqueda de hiperparametros mediante GridSearch

In [155]:
clf_svc = svm.SVC(kernel='rbf')

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10, 50],
    'gamma': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'gamma', 'score', 'holdout_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits


,encoding,scaling,selection,C,gamma,score,holdout_score
0,onehot,standard,var_thres,50,0.05,0.826562,0.826087
1,onehot,minmax,var_thres,10,1.00,0.829688,0.807453
2,onehot,robust,var_thres,10,0.10,0.818750,0.838509
3,onehot,power,var_thres,10,0.05,0.829688,0.819876
4,onehot,normalizer,var_thres,50,2.00,0.804688,0.788820


---

#### 3.3.4 Sigmoide

Entrenamiento con valores por defecto

In [40]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='sigmoid')
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.726141
1,onehot,minmax,var_thres,0.684647
2,onehot,robust,var_thres,0.643154
3,onehot,power,var_thres,0.709544
4,onehot,normalizer,var_thres,0.655602


---

Busqueda de hiperparametros mediante GridSearch

In [156]:
clf_svc = svm.SVC(kernel='sigmoid')

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'gamma': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0],
    'coef0': [0.0, 0.5, 1.0]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'coef0', 'gamma', 'score', 'holdout_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits


,encoding,scaling,selection,C,coef0,gamma,score,holdout_score
0,onehot,standard,var_thres,0.1,0.5,0.10,0.789062,0.732919
1,onehot,minmax,var_thres,0.1,0.0,0.10,0.796875,0.795031
2,onehot,robust,var_thres,0.1,0.5,0.10,0.735938,0.689441
3,onehot,power,var_thres,0.1,0.5,0.05,0.790625,0.795031
4,onehot,normalizer,var_thres,10.0,0.0,0.50,0.712500,0.683230


---

### 3.4 Regresor Lineal (Logistic Regression)

In [75]:
from sklearn.linear_model import LogisticRegression

In [76]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_logreg = LogisticRegression()
    clf_logreg.fit(conjunto[0], conjunto[2])
    pred = clf_logreg.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.804979
1,onehot,minmax,var_thres,0.796680
2,onehot,robust,var_thres,0.800830
3,onehot,power,var_thres,0.800830
4,onehot,normalizer,var_thres,0.663900


---

Busqueda de Hiperparámetros mediante GridSearch

In [79]:
logreg_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'penalty': ['l1','l2','elasticnet','none'],
}

gs = model_selection.GridSearchCV(
    clf_logreg,
    logreg_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

In [81]:
metricas = []
for conjunto in datos:
    
    gs_results = gs.fit(conjunto[0], conjunto[2])
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    for key in gs_results.best_params_:
        row.append(gs_results.best_params_[key])
    row.append(gs_results.best_score_)
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection','C', 'penalty', 'score'])
dd

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.8125            nan 0.81071429        nan 0.81785714
        nan 0.81071429        nan 0.80892857        nan 0.81071429
        nan 0.81071429        nan 0.81071429]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.72857143        nan 0.81071429        nan 0.81785714
        nan 0.81071429        nan 0.8125            nan 0.81071429
        nan 0.8125            nan 0.81071429]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.725             nan 0.81071429        nan 0.81964286
        nan 0.81071429        nan 0.81071429        nan 0.81071429
        nan 0.81

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.79107143        nan 0.79464286        nan 0.80357143
        nan 0.79464286        nan 0.79464286        nan 0.79464286
        nan 0.79464286        nan 0.79464286]
  category=UserWarning


Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.61785714        nan 0.79285714        nan 0.62142857
        nan 0.79285714        nan 0.67857143        nan 0.79285714
        nan 0.73214286        nan 0.79285714]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:1323: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/home/javier/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

,encoding,scaling,selection,C,penalty,score
0,onehot,standard,var_thres,0.10,l2,0.817857
1,onehot,minmax,var_thres,0.10,l2,0.817857
2,onehot,robust,var_thres,0.10,l2,0.819643
3,onehot,power,var_thres,0.10,l2,0.803571
4,onehot,normalizer,var_thres,0.01,none,0.792857


### 3.5 Decision Tree

Entrenamiento con valores por defecto

In [165]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_tree = tree.DecisionTreeClassifier(random_state=13)
    clf_tree.fit(conjunto[0], conjunto[2])
    pred = clf_tree.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.807453
1,onehot,minmax,var_thres,0.807453
2,onehot,robust,var_thres,0.813665
3,onehot,power,var_thres,0.801242
4,onehot,normalizer,var_thres,0.770186


---

Busqueda de Hiperparámetros mediante GridSearch

In [163]:
clf_tree = tree.DecisionTreeClassifier(random_state=13)

tree_grid_params = {
    'max_depth': np.arange(1,32),
    'min_samples_leaf': np.arange(1,32),
    'criterion': ["gini","entropy"]
}

gs = model_selection.GridSearchCV(
    clf_tree,
    tree_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','criterion', 'max_depth', 'min_samples_leaf', 'score', 'holdout_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits


,encoding,scaling,selection,criterion,max_depth,min_samples_leaf,score,holdout_score
0,onehot,standard,var_thres,gini,8,3,0.815625,0.826087
1,onehot,minmax,var_thres,gini,6,2,0.817187,0.826087
2,onehot,robust,var_thres,entropy,4,10,0.815625,0.788820
3,onehot,power,var_thres,gini,6,2,0.817187,0.819876
4,onehot,normalizer,var_thres,gini,4,10,0.818750,0.795031


---

### 3.6 Redes Neuronales

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

### 3.7 Ensambles

In [45]:
from sklearn import ensemble

#### 3.7.1 Bagging

In [47]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_bagging = ensemble.BaggingClassifier()
    clf_bagging.fit(conjunto[0], conjunto[2])
    pred = clf_bagging.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.829876
1,onehot,minmax,var_thres,0.821577
2,onehot,robust,var_thres,0.809129
3,onehot,power,var_thres,0.829876
4,onehot,normalizer,var_thres,0.817427


---

#### 3.7.2 RandomForest

In [49]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    random_forest = ensemble.RandomForestClassifier()
    random_forest.fit(conjunto[0], conjunto[2])
    pred = random_forest.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.817427
1,onehot,minmax,var_thres,0.809129
2,onehot,robust,var_thres,0.813278
3,onehot,power,var_thres,0.809129
4,onehot,normalizer,var_thres,0.838174


#### 3.7.3 Gradient Boost

In [51]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_boosting = ensemble.GradientBoostingClassifier()
    clf_boosting.fit(conjunto[0], conjunto[2])
    pred = clf_boosting.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.804979
1,onehot,minmax,var_thres,0.800830
2,onehot,robust,var_thres,0.804979
3,onehot,power,var_thres,0.800830
4,onehot,normalizer,var_thres,0.825726


#### 3.7.4 Stacking

In [62]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_1 = neighbors.KNeighborsClassifier()
    clf_2 = ensemble.RandomForestClassifier()
    stacker = ensemble.StackingClassifier(estimators=[('clf_1', clf_1), ('clf_2', clf_2)])
    
    stacker.fit(conjunto[0], conjunto[2])
    pred = stacker.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.792531
1,onehot,minmax,var_thres,0.800830
2,onehot,robust,var_thres,0.825726
3,onehot,power,var_thres,0.804979
4,onehot,normalizer,var_thres,0.804979


#### 3.7.5 Voting

In [69]:
clf_1 = neighbors.KNeighborsClassifier()
clf_2 = tree.DecisionTreeClassifier()
clf_3 = svm.SVC()

In [70]:
voting = ensemble.VotingClassifier(estimators=[
    ('clf_1', clf_1),
    ('clf_2', clf_2),
    ('clf_3', clf_3)
])

In [72]:
voting.fit(X_train_a, y_train_a)
pred = voting.predict(X_test_a)
score = (y_test_a == pred)
    
score.mean()

0.7925311203319502

In [74]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_1 = neighbors.KNeighborsClassifier()
    clf_2 = tree.DecisionTreeClassifier()
    clf_3 = svm.SVC()
    voting = ensemble.VotingClassifier(estimators=[
        ('clf_1', clf_1),
        ('clf_2', clf_2),
        ('clf_3', clf_3)
    ])
    
    voting.fit(conjunto[0], conjunto[2])
    pred = voting.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.792531
1,onehot,minmax,var_thres,0.780083
2,onehot,robust,var_thres,0.804979
3,onehot,power,var_thres,0.792531
4,onehot,normalizer,var_thres,0.780083


#### 3.7.6 Cascading

In [85]:
#ver especificamente en otro momento